In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Загружаем данные и сразу переводим общие столбцы в двух датасетах к одному типу данных

heart_path = "heart (1).csv"
cardio_path = "baseline/daniil_vasilev_baseline_cml/data/cardio_train_correct.parquet"

heart_data = pd.read_csv(heart_path)
heart_data.columns = heart_data.columns.str.lower()
heart_data['cholesterol_int'] = heart_data['cholesterol'].copy()  # так как в одной из моделей нужен этот признак в числовом виде, а не булевом
heart_data['cholesterol'] = heart_data['cholesterol'].apply(lambda value: 0 if value <= 210 else 1)  # а в другой модели нужно булевое значение его

cardio_data = pd.read_parquet(cardio_path).drop('id', axis=1)
cardio_data['age'] = cardio_data['age'].astype(int)
cardio_data = cardio_data.rename(columns={'cardio': 'target'})
heart_data.shape, cardio_data.shape

((1025, 15), (70000, 12))

In [6]:
full_df = heart_data.merge(cardio_data, on=['age', 'cholesterol', 'target']).reset_index(drop=True)
print(f'Получили {full_df.shape[0]} строк после объединения двух датасетов')
full_df.head()

Получили 410127 строк после объединения двух датасетов


,age,sex,cheastpaintype,restingbp,cholesterol,fastingbs,restingecg,maxhr,exerciseangina,oldpeak,...,cholesterol_int,gender,height,weight,ap_hi,ap_lo,gluc,smoke,alco,active
0,52,1,0,125,1,0,1,168,0,1.0,...,212,0,167.000000,80.0,190,90,0,0,1,0
1,52,1,0,125,1,0,1,168,0,1.0,...,212,0,177.980827,63.0,110,70,0,0,0,1
2,52,1,0,125,1,0,1,168,0,1.0,...,212,1,173.000000,75.0,130,80,0,1,1,0
3,52,1,0,125,1,0,1,168,0,1.0,...,212,0,167.000000,70.0,110,70,0,0,0,0
4,52,1,0,125,1,0,1,168,0,1.0,...,212,0,163.000000,63.0,120,80,1,0,0,1


`После объединения датасетов смогли получить 410 тысяч объектов с размеченным таргетом.`

In [7]:
# произведем оверсемплинг с помощью SMOTE
from imblearn.over_sampling import SMOTE

X, y = full_df.drop('target', axis=1), full_df['target']
smote = SMOTE(sampling_strategy='all', k_neighbors=5, random_state=42)
X_sm, y_sm = smote.fit_resample(X, y)

In [8]:
print(f'Было {X.shape[0]} объектов')
print(f'Стало {X_sm.shape[0]} объектов')

Было 410127 объектов
Стало 446388 объектов


`С помощью оверсемплинга смогли сбалансировать классы в данных, получив дополнительно новые 36 тысяч объектов.`

In [9]:
# разделим данные на обучающую выборку и тестовую
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.25, random_state=42)

In [10]:
# разделим признаки, которые использовались по отдельным моделям
heart_features = heart_data.columns.to_list().copy()
heart_features.remove('target')

cardio_features = cardio_data.columns.to_list().copy()
cardio_features.remove('target')

#### Бейзлайн модель датасета cardio_data (Дани)

In [11]:
xgb_params = {
            "n_estimators": 500,
            "learning_rate": 0.1,
            "max_depth": 3,
            "objective": "binary:logistic",
            "eval_metric": "logloss",
            "random_state": 12,
            "n_splits": 4
}

X_train_train, X_val, y_train_train, y_val = train_test_split(X_train[cardio_features], y_train, test_size=0.25, random_state=42)
model_xgb = XGBClassifier(**xgb_params)
model_xgb.fit(X_train_train, y_train_train, eval_set=[(X_val, y_val)], verbose=False)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=500,
              n_jobs=None, n_splits=4, num_parallel_tree=None, ...)

In [12]:
pred_xgb = model_xgb.predict(X_test[cardio_features])
auc_xgb = roc_auc_score(y_test, pred_xgb)
print(f'ROC-AUC модели XGBoost = {round(auc_xgb, 3)}')

ROC-AUC модели XGBoost = 0.825


#### Бейзлайн модель датасета heart_data (Вани)

In [16]:
heart_X = X_train[heart_features].drop(columns=['cholesterol']).rename(columns={'cholesterol_int': 'cholesterol'})
model_lr = LogisticRegression(max_iter=1000, random_state=42)
model_lr.fit(heart_X, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [18]:
heart_X_test = X_test[heart_features].drop(columns=['cholesterol']).rename(columns={'cholesterol_int': 'cholesterol'})
pred_lr = model_lr.predict(heart_X_test)
auc_lr = roc_auc_score(y_test, pred_lr)
print(f'ROC-AUC модели логистической регрессии = {round(auc_lr, 3)}')

ROC-AUC модели логистической регрессии = 0.848


#### Ансамблируем

In [20]:
# с порогом из безлайна - 0.5
prob_xgb = model_xgb.predict_proba(X_test[cardio_features])[:, 1]
prob_lr = model_lr.predict_proba(heart_X_test)[:, 1]
ansambl_pred = (prob_xgb + prob_lr) / 2
THRESHOLD = 0.5
final_pred = [1 if val >= THRESHOLD else 0 for val in ansambl_pred]
auc_ansambl = roc_auc_score(y_test, final_pred)
print(f'ROC-AUC ансамбля с порогом по умолчанию = {round(auc_ansambl, 3)}')

ROC-AUC ансамбля с порогом по умолчанию = 0.911


In [22]:
# будем перебирать порог на валидационной выборке
X_train_train, X_val, y_train_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
results = []
for THRESHOLD in np.arange(0.1, 1.01, 0.1):
    heart_X_val = X_val[heart_features].drop(columns=['cholesterol']).rename(columns={'cholesterol_int': 'cholesterol'})
    prob_xgb = model_xgb.predict_proba(X_val[cardio_features])[:, 1]
    prob_rf = model_lr.predict_proba(heart_X_val)[:, 1]
    ansambl_pred = (prob_xgb + prob_rf) / 2
    final_pred = [1 if val >= THRESHOLD else 0 for val in ansambl_pred]
    auc = roc_auc_score(y_val, final_pred)
    accuracy = accuracy_score(y_val, final_pred)
    results.append((THRESHOLD, auc, accuracy))

results_df = pd.DataFrame(results, columns=['threshold', 'roc_auc', 'accuracy']).sort_values(by='roc_auc', ascending=False)
results_df

,threshold,roc_auc,accuracy
4,0.5,0.910059,0.910129
3,0.4,0.895082,0.895517
5,0.6,0.885255,0.884800
2,0.3,0.841473,0.842386
6,0.7,0.839379,0.838479
1,0.2,0.777867,0.779266
7,0.8,0.752661,0.751129
8,0.9,0.657388,0.655189
0,0.1,0.648504,0.650768
9,1.0,0.500000,0.496774


`Видим, что наилучшее качество ансамбля достигается при пороге в 0.5. ROC-AUC и accuracy равен 0.910, что весьма неплохо для бейзлайн-ансамбля.`